In [17]:
import os
import cv2
import numpy as np

# Define the directory paths
RAW_DATA_DIR = "C:/Users/dunli/Downloads/DeepBacs_Data_Object_Detection_E.coli_Growth_Stage/image_patches/train_images"
AUG_DATA_DIR = "C:/Users/dunli/Downloads/augmented_data"

# Define augmentation parameters
num_of_images = 100

# Define augmentation functions
def crop(image):
    h, w = image.shape[:2]
    start_row, start_col = int(h * 0.1), int(w * 0.1)
    end_row, end_col = int(h * 0.9), int(w * 0.9)
    cropped = image[start_row:end_row, start_col:end_col]
    return cropped

def flip(image):
    flipped = cv2.flip(image, 1)
    return flipped

def rotate(image):
    angle = np.random.randint(0, 360)
    rows, cols = image.shape[:2]
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    rotated = cv2.warpAffine(image, M, (cols, rows))
    return rotated

def scale(image):
    scale_factor = np.random.uniform(0.8, 1.2)
    scaled = cv2.resize(image, None, fx=scale_factor, fy=scale_factor)
    return scaled

def translate(image):
    rows, cols = image.shape[:2]
    x_translation = np.random.randint(-50, 50)
    y_translation = np.random.randint(-50, 50)
    M = np.float32([[1, 0, x_translation], [0, 1, y_translation]])
    translated = cv2.warpAffine(image, M, (cols, rows))
    return translated

def adjust_contrast(image):
    alpha = np.random.uniform(0.5, 1.5)
    beta = np.random.uniform(-50, 50)
    adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return adjusted

# Create augmented data directory if it doesn't exist
os.makedirs(AUG_DATA_DIR, exist_ok=True)

# Loop through all images in the raw data directory
for root, dirs, files in os.walk(RAW_DATA_DIR):
    for file in files:
        if file.endswith(".png"):
            image_path = os.path.join(root, file)
            image = cv2.imread(image_path)

            # Apply augmentation functions
            augmented_images = []
            augmented_images.append(crop(image))
            augmented_images.append(flip(image))
            augmented_images.append(rotate(image))
            augmented_images.append(scale(image))
            augmented_images.append(translate(image))
            augmented_images.append(adjust_contrast(image))

            # Save augmented images
            basename = os.path.basename(image_path).split('.')[0]
            for i, augmented_image in enumerate(augmented_images):
                cv2.imwrite(os.path.join(AUG_DATA_DIR, f"{basename}_aug_{i}.png"), augmented_image)
